In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [ ]:
pre_fine_tuning = load_model("../TumorSense_app/model.h5")

In [ ]:
pro_fine_tuning = load_model("../TumorSense_app/model.h5")

In [ ]:
index = ['glioma','meningioma','normal','adenoma']

### fine tune model

In [ ]:

dataset_dir = '../../Datasets/brain/Training'

model = load_model('../TumorSense_app/model.h5')
num_classes = len(index)

x = model.layers[-2].output
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

fine_tuned_model = Model(inputs=model.input, outputs=predictions)

for layer in fine_tuned_model.layers[:-5]:
    layer.trainable = False

fine_tuned_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
history = fine_tuned_model.fit(test_generator, epochs=10)

test_loss, test_acc = fine_tuned_model.evaluate(test_generator)
print('Test accuracy:', test_acc)

In [ ]:
# fine_tuned_model.save('../TumorSense_app/fine_tuned_model.h5')